# Benchmark ReQA
- Input: `06_calculate_distance.parquet`

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from utils import encode_in_batch

import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

2023-08-30 16:18:04.414649: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-30 16:18:04.439232: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 16:18:04.729101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
data = pd.read_parquet('data/06_calculate_distance.parquet')
data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,dis_ltw2v_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_thai2fit_aug,dis_thai2trans_aug,dis_wordnet_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.379822,0.457964,0.442132,0.442132,0.664607,0.373971,0.391374,0.213284,0.000000,9.606189e-02
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.028428,0.512295,0.604564,0.552465,0.590820,0.583206,0.645776,0.232190,0.000000,0.000000e+00
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.017806,0.408607,0.408607,0.346143,0.346143,0.328919,0.328919,0.150410,0.000000,0.000000e+00
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.044393,0.600420,0.717404,0.606477,0.717404,0.433692,0.523367,0.155829,0.009204,1.820072e-01
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.180941,0.238710,0.388979,0.151006,0.443043,0.097685,0.097685,0.150196,0.156331,8.621460e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.038205,0.459041,0.345795,0.330124,0.345795,0.363513,0.284700,0.063899,0.012518,1.700147e-01
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เนชยมีฟิตี้07ต เปิดตัวครั้งแรกเม...,BioShock Infinite เปิดตัวครั้งแรกเมื่อใด?,การเปิดตัวครั้งแรกของ BioShock Infinite คือเมื...,ไบโอช็อค อินฟินิต เผยโฉมเป็นครั้งแรกเมื่อไร?,...,0.168088,0.609325,0.713147,0.475034,0.686884,0.482104,0.602809,0.097

In [4]:
# Add None so that the first benchmark only consists of the original questions
all_augment_cols = [None] + [col for col in data.columns if col.startswith('th_')]
all_augment_cols

[None,
 'th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_ltw2v_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_thai2fit_aug',
 'th_thai2trans_aug',
 'th_wordnet_aug']

In [5]:
model_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

2023-08-30 16:18:05.929850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-30 16:18:05.944769: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
def evaluate(
    question_id, question_all, context_id, context_all, mrr_rank=10, status=True
):
    top_1 = 0
    top_5 = 0
    top_10 = 0
    mrr_score = 0
    context_id = np.array(context_id)
    sim_score = np.inner(question_all, context_all)
    if status == True:
        status_bar = enumerate(tqdm(sim_score))
    else:
        status_bar = enumerate(sim_score)
    for idx, sim in status_bar:
        index = np.argsort(sim)[::-1]
        index_edit = [context_id[x] for x in index]
        try:
            idx_search = list(index_edit).index(question_id[idx])
        except:
            idx_search = 999999
        if idx_search == 0:
            top_1 += 1
            top_5 += 1
            top_10 += 1
        elif idx_search < 5:
            top_5 += 1
            top_10 += 1
        elif idx_search < 10:
            top_10 += 1
        if idx_search < mrr_rank:
            mrr_score += 1 / (idx_search + 1)
    return (
        top_1 / len(question_all),
        top_5 / len(question_all),
        top_10 / len(question_all),
        mrr_score / len(question_all),
    )

In [7]:
def benchmark_single(data, question_col="question", aug_col=None, model_embed=model_embed, filter_zero_dist=True):
    # Check if column has a distance counterpart
    questions = data[question_col].values

    question_all = encode_in_batch(model_embed, questions)
    context_all = encode_in_batch(model_embed, data["context"].values)

    question_ids = data["id"].values
    context_ids = data["id"].values

    if aug_col:
        augments = data[aug_col].values

        if filter_zero_dist:
            distances = data[aug_col.replace("th_", "dis_")].values
            augments = augments[distances > 0]
            augment_ids = data["id"].values[distances > 0]
        else:
            augment_ids = data["id"].values
        
        augment_all = encode_in_batch(model_embed, augments)
        
        # Augment into context
        context_all = np.concatenate([context_all, augment_all], axis=0)
        context_ids = np.concatenate([context_ids, augment_ids], axis=0)

    top_1, top_5, top_10, mrr_score = evaluate(
        question_ids, question_all, context_ids, context_all, status=True
    )
    return top_1, top_5, top_10, mrr_score

In [8]:
results = []

for col in tqdm(all_augment_cols):
    top_1, top_5, top_10, mrr_score = benchmark_single(data, aug_col=col)
    print(f"{col}: top_1: {top_1}, top_5: {top_5}, top_10: {top_10}, mrr_score: {mrr_score}")
    results.append({"name": col, "top_1": top_1, "top_5": top_5, "top_10": top_10, "mrr_score": mrr_score})

results = pd.DataFrame(results)
results

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

None: top_1: 0.14776207302709068, top_5: 0.41707891637220257, top_10: 0.5162544169611307, mrr_score: 0.2575784770131067


...

th_thai2trans_aug: top_1: 0.7710836277974087, top_5: 0.7896348645465253, top_10: 0.7977031802120141, mrr_score: 0.7798150718866277


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

th_wordnet_aug: top_1: 0.6121908127208481, top_5: 0.6481743227326267, top_10: 0.6597173144876325, mrr_score: 0.6284836969730977


,name,top_1,top_5,top_10,mrr_score
0,None,0.147762,0.417079,0.516254,0.257578
1,th_aug,0.660012,0.813015,0.851060,0.725883
2,th_fasttext_aug,0.605183,0.799117,0.851590,0.688540
3,th_llm_gec_aug,0.611837,0.782273,0.827503,0.684874
4,th_llm_paraphrase_aug,0.582686,0.765135,0.813074,0.661460
5,th_ltw2v_aug,0.730506,0.816196,0.837044,0.767637
6,th_qcpg_0.2_aug,0.316843,0.530801,0.607538,0.407302
7,th_qcpg_0.2_llm_gec_aug,0.249352,0.454594,0.538987,0.337123
8,th_qcpg_0.5_aug,0.341637,0.554476,0.631037,0.432222
9,th_qcpg_0.5_llm_gec_aug,0.277326,0.485395,0.569435,0.366687


In [ ]:
results.to_csv("results/07_benchmark_reqa.csv", index=False)